In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
code="""

#include<stdio.h>
#include<omp.h>
#include<time.h>
#include<stdlib.h>

#define block 10

void pbs(int *a,int key,int start,int end);

int bs(int *a,int key,int start,int end)
{
    int res=-1;
    if(start<=end)
    {
        int mid=(start+end)/2;
        
        if(a[mid]==key)
        {
            res=mid;
        }
        else if(a[mid]<key)
        {
            res=bs(a,key,mid+1,end);
        }
        else
        {
            res=bs(a,key,start,mid-1);
        }
    }    
    return res;
}

void pbs(int *a,int key,int start,int end)
{
    int res[4];
    omp_set_num_threads(4);
    #pragma omp parallel for shared(res,a,key)
    for(int i=0;i<4;i++)
    {
        res[i]=bs(a,key,i*block,(i+1)*block-1);
    }
    /*for(int i=0;i<4;i++)
    {
        printf("%d\t",res[i]);
    }*/
}

int main()
{
  int *a;
  int n=40;
  a=(int *)malloc(sizeof(int)*n);
  
  for(int i=0;i<n;i++)
    a[i]=i;
    
  double start,end;
  
  start=omp_get_wtime();
  int res=bs(a,39000,0,n-1);
  printf("%d\t",res);
  end=omp_get_wtime();
  
  printf("%f\t",end-start);
  
  start=omp_get_wtime();
  pbs(a,39000,0,n-1);
  end=omp_get_wtime();
  
  printf("%f\t",end-start);
}

"""

In [0]:
text_file = open('search.c',"w")
text_file.write(code)
text_file.close()

In [0]:
!gcc -fopenmp search.c

In [0]:
!./a.out

-1	0.000017	0.000160	

In [0]:
code1="""

#include<omp.h>
#include<chrono>
#include<iostream>
#include<stdio.h>

using namespace std;

class Node
{
  public:
    int val;
    Node *left,*right;
    
    Node(int val)
    {
        this->val=val;
        this->left=NULL;
        this->right=NULL;
    }
};

void dfs(Node *root)
{
    if(root==NULL)
      return;
    else
    {
        dfs(root->left);
        
        dfs(root->right);
        
        cout<<root->val<<"\t";
    }
}

void pdfs(Node *root)
{
    if(root==NULL)
      return;
    
    #pragma omp parallel sections
    {
        #pragma omp section
        {
            pdfs(root->left);
        } 
        
        #pragma omp section
        {
            pdfs(root->right);
        }
    }
    
    cout<<root->val<<"\t";
    
    
    
}

int main()
{
  int n=5000;
  int a[n];
  
  for(int i=0;i<n;i++)
    a[i]=i;
  
  Node *root=NULL;
  
  root=new Node(a[0]);
  
  for(int i=1;i<n;i++)
  {
      Node *nn=new Node(a[i]);
      Node *temp=root;
      
      while(temp!=NULL)
      {
        if(nn->val < temp->val)
        {
            if(temp->left==NULL)
            {
                temp->left=nn;
                break;
                
            }
            else
              temp=temp->left;
        }
        else
        {
            if(temp->right==NULL)
            {
                temp->right=nn;
                break;
            }
            else
              temp=temp->right;
        }
      }      
  } //end for
  
  double start,end;
  start=omp_get_wtime();
  dfs(root);
  end=omp_get_wtime();
  cout<<endl;
  cout<<"serial:"<<end-start<<endl;
  
  start=omp_get_wtime();
  pdfs(root);
  end=omp_get_wtime();
  cout<<endl;
  cout<<"parallel:"<<end-start<<endl;
}

"""

In [0]:
text_file = open('dfs.cpp',"w")
text_file.write(code1)
text_file.close()

In [0]:
!g++ -fopenmp dfs.cpp

In [0]:

!./a.out

4999	4998	4997	4996	4995	4994	4993	4992	4991	4990	4989	4988	4987	4986	4985	4984	4983	4982	4981	4980	4979	4978	4977	4976	4975	4974	4973	4972	4971	4970	4969	4968	4967	4966	4965	4964	4963	4962	4961	4960	4959	4958	4957	4956	4955	4954	4953	4952	4951	4950	4949	4948	4947	4946	4945	4944	4943	4942	4941	4940	4939	4938	4937	4936	4935	4934	4933	4932	4931	4930	4929	4928	4927	4926	4925	4924	4923	4922	4921	4920	4919	4918	4917	4916	4915	4914	4913	4912	4911	4910	4909	4908	4907	4906	4905	4904	4903	4902	4901	4900	4899	4898	4897	4896	4895	4894	4893	4892	4891	4890	4889	4888	4887	4886	4885	4884	4883	4882	4881	4880	4879	4878	4877	4876	4875	4874	4873	4872	4871	4870	4869	4868	4867	4866	4865	4864	4863	4862	4861	4860	4859	4858	4857	4856	4855	4854	4853	4852	4851	4850	4849	4848	4847	4846	4845	4844	4843	4842	4841	4840	4839	4838	4837	4836	4835	4834	4833	4832	4831	4830	4829	4828	4827	4826	4825	4824	4823	4822	4821	4820	4819	4818	4817	4816	4815	4814	4813	4812	4811	4810	4809	4808	4807	4806	4805	4804	4803	4802	4801	4800	

In [0]:
codef="""

#include<cstdio>
#include<mpi.h>
#include<iostream>
#include<chrono>
#include<unistd.h>

#include <time.h>

using namespace std::chrono;
using namespace std;

void binary_search(int a[],int start,int end,int key,int rank)
{
	while(start <= end)
	{
		int m = (start+end)/2;
        
		if(a[m]==key)
		{
			cout<<"The element is found by Process No "<<rank+1<<endl;
			return;
		}
		else if(a[m]<key)
		{
			start=m+1;
		}
		else
		{
			end=m-1;
		}
	}
    cout<<"Not found by Process No : "<<rank+1<<endl;
}

int main(int argc, char **argv)
{
    int n=8000;
    int key=4500;
    double c[4];
    int a[n];
    for(int i=0;i<n;i++)
    {
          a[i]=i+1;
    }
    
	int rank,blocksize;
	MPI_Init(&argc,&argv); //Initialise MPI
	
	MPI_Comm_rank(MPI_COMM_WORLD,&rank);// Sets the rank. The rank of a process always ranges from o to size-1

	MPI_Comm_size(MPI_COMM_WORLD,&blocksize); // Sets the size. size = no. of processes
	
	blocksize=n/4; // to allocate uniform load on each of the 4 processors.
	
	double start = MPI_Wtime();//MPI_Wtime() : returns current time.

	binary_search(a,rank*blocksize,(rank+1)*blocksize-1,key,rank);

	double end = MPI_Wtime();

	cout<<"The time for process " <<rank+1<<"is "<<(end-start)*1000<<endl<<endl;
	c[rank]=end;

	MPI_Finalize(); //Finalize MPI

}

"""

In [0]:
text_file = open('search.cpp',"w")
text_file.write(codef)
text_file.close()

In [0]:
!mpicxx search.cpp

In [0]:
!mpirun --allow-run-as-root -np 4 ./a.out

Not found by Process No : 1
The time for process 1is 0.044048

Not found by Process No : 4
The time for process 4is 0.062787

The element is found by Process No 3
The time for process 3is 0.067513

Not found by Process No : 2
The time for process 2is 0.092995

